In [1]:
import sys

import nvidia.cudnn
print(nvidia.cudnn.__file__)
sys.path.append('/global/homes/a/achen899/.local/perlmutter/3.9-anaconda-2021.11/lib/python3.9/site-packages/')


/global/homes/a/achen899/.conda/envs/env_gan/lib/python3.9/site-packages/nvidia/cudnn/__init__.py


In [ ]:
from hadronic_to_root import convert
import pprint
import awkward as ak
import uproot

convert("/global/cfs/projectdirs/m3443/data/ForHadronic/train_data/hadron_pi_mode2.csv", "/global/cfs/projectdirs/m3443/data/ForHadronic/train_data/pimode/hadron_pi_mode.root")

2023-09-11 13:44:13.217769: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-11 13:44:19.489271: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


True000


In [4]:
import uproot

out_branch_names = ['particle_id', "particle_E", "particle_px", "particle_py", "particle_pz"]
in_branch_names = ['incoming']
branch_names = out_branch_names + in_branch_names

f = uproot.open("/global/cfs/projectdirs/m3443/data/ForHadronic/train_data/pimode/hadron_pi_mode.root")
tree = f["output"]
all_dict = tree.arrays(branch_names, library="np")
  
print(len(all_dict['incoming']))
print(len(all_dict['particle_id']))

3000000
3000000


In [4]:
#import pypdt
import numpy as np
import math
import os
import sys
import time

cond_vectors, truth_vectors = np.zeros(5), np.zeros(5)

scale_max = 0
scale_min = 0
particle_type_count = 1
partdict = dict()
count=1
for i, cond in enumerate(df["incoming"]):
    remaining_in_vector = cond
    scale_max = max(scale_max, cond[0])
    for j, o_energy in enumerate(df["particle_E"][i]):
        if df["particle_id"][i][j] in partdict.keys():
            truth_vector = np.array([o_energy, df["particle_px"][i][j], df["particle_py"][i][j], df["particle_pz"][i][j], partdict[df["particle_id"][i][j]]])
        else:
            partdict[df["particle_id"][i][j]] = particle_type_count
            truth_vector = np.array([o_energy, df["particle_px"][i][j], df["particle_py"][i][j], df["particle_pz"][i][j], partdict[df["particle_id"][i][j]]])
            particle_type_count += 1
        #print(truth_vector[4], end =" ")
        cond_vector = np.append(remaining_in_vector, [j])
        cond_vectors = np.vstack((cond_vectors, cond_vector))
        truth_vectors = np.vstack((truth_vectors, truth_vector))
        remaining_in_vector = remaining_in_vector - truth_vector[:4]
    scale_min = min(scale_min, remaining_in_vector[0])
    print(count, end="\r", flush=True)
    count += 1
ecv_i = cond_vectors[1:,4]
etv_id = truth_vectors[1:,4]

truth_vectors_no_id = truth_vectors[1:,:4]
cond_vectors_no_i = cond_vectors[1:,:4]
truth_vectors_no_id = pscale(truth_vectors_no_id, scale_max, scale_min)
cond_vectors_no_i = pscale(cond_vectors_no_i, scale_max, scale_min)

cond_vectors = np.concatenate((cond_vectors_no_i, ecv_i[:, None]), axis=1)
truth_vectors = np.concatenate((etv_id[:, None], truth_vectors_no_id), axis=1)

print(np.shape(truth_vectors))
print(np.shape(cond_vectors))

(1634977, 5)
(1634977, 5)
